# Chatbot with LangChain conversational chain and Mistral 🤖💬

이 노트북에서는 사업주의 정책과 같은 사용자 정의 데이터에 대한 질문에 응답할 수 있는 챗봇을 구축하겠습니다.

[Mistral]() LLM은 apache-2.0 license로 자유롭게 이용할 수 있으며, 별도의 과금이 없습니다. 

여기서는 [openbuddy-mistral-7B-v13-GGUF](https://huggingface.co/TheBloke/openbuddy-mistral-7B-v13-GGUF)모델은 Mistral 모델을 한국어등 다국어를 지원할 수 있도록 파인튜닝한 모델입니다.

챗봇은 LangChain의 `ConversationalRetrievalChain`을 사용하며 다음과 같은 기능을 갖습니다.

- 자연어로 묻는 질문에 답변
- Elasticsearch에서 하이브리드 검색을 실행하여 질문에 답하는 문서를 찾으세요.
- Mistral LLM을 활용하여 답변 추출 및 요약
- 후속 질문을 위한 대화 기억 유지


## Requirements 🧰

이 예에서는 다음이 필요합니다.

- Python 3.6 이상
- 로컬에 설치된 Elasticsearch

## Install packages 📦

먼저 이 예제에 필요한 패키지를 `pip install`합니다.


In [ ]:
%pip install -U langchain elasticsearch sentence_transformers llama-cpp-python wget

## Initialize clients 🔌

다음으로 `getpass`를 사용하여 자격 증명을 입력합니다. `getpass`는 Python 표준 라이브러리의 일부이며 자격 증명을 안전하게 요청하는 데 사용됩니다.

In [ ]:
from getpass import getpass

ES_URL = input('Elasticsearch URL(ex:https://127.0.0.1:9200): ')
ES_USER = "elastic"
ES_USER_PASSWORD = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch pem 파일 경로: ')
# pem 생성 방법: https://cdax.ch/2022/02/20/elasticsearch-python-workshop-1-the-basics/

# set OpenAI API key
# OPENAI_API_KEY = getpass("OpenAI API key")


## Load and process documents 📄

데이터를 로드할 시간입니다!   
우리는 직원 문서 및 정책 목록인 직장 검색 예제 데이터를 사용할 것입니다.


In [ ]:
import json
from urllib.request import urlopen
import os

cwd = os.getcwd()
url = cwd + "/data/workplace-docs.json"
response = open(url)

workplace_docs = json.loads(response.read())

print(f"Successfully loaded {len(workplace_docs)} documents")

## Chunk documents into passages 🪓

봇과 채팅하는 동안 봇은 관련 문서를 찾기 위해 인덱스에서 시멘틱 검색을 실행합니다.   
이것이 정확하려면 전체 문서를 작은 청크(chunk) -구절(passage)이라고도 함-로 분할해야 합니다.   
이런 방식으로 의미론적 검색은 문서 내에서 우리의 질문에 가장 답할 가능성이 높은 구절을 찾을 것입니다.

우리는 LangChain의 `CharacterTextSplitter`를 사용하고 문서의 텍스트를 청크 사이에 약간 겹치도록 800자로 분할할 것입니다.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["content"])
    metadata.append({
        "name": doc["name"],
        "summary": doc["summary"]
    })

text_splitter = CharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=256
)
docs = text_splitter.create_documents(content, metadatas=metadata)

print(f"Split {len(workplace_docs)} documents into {len(docs)} passages")

In [ ]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    [ES_URL],
    basic_auth=(ES_USER, ES_USER_PASSWORD),
    ca_certs=CERT_PATH
)


In [ ]:

if client.indices.exists(index="workplace-docs"):
    client.indices.delete(index="workplace-docs")

임베딩을 생성하고 이를 사용하여 문서를 인덱싱해 보겠습니다.


In [ ]:
import os
cwd = os.getcwd()

if os.path.isdir(cwd + "/models"):
    pass
else:
    os.mkdir(cwd + "/models")

In [ ]:
os.chdir(cwd + "/models")

try :
    os.system("git lfs install & git clone https://huggingface.co/intfloat/multilingual-e5-base")
except:
    print('이미 모델이 존재합니다.')

os.chdir(cwd)

In [ ]:
from langchain.vectorstores import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings

print(cwd + "/models/multilingual-e5-base")

embeddings = HuggingFaceEmbeddings(model_name=cwd + "/models/multilingual-e5-base", model_kwargs = {'device': 'cpu'} )

vector_store = ElasticsearchStore.from_documents(
    docs,
    es_connection = client,
    index_name="workplace-docs",
    embedding=embeddings
)

## Chat with the chatbot 💬

챗봇을 초기화해 보겠습니다.   
Elasticsearch를 문서 검색 및 채팅 세션 기록 저장을 위한 저장소로 정의하고,   
Mistral를 질문을 해석하고 답변을 요약하는 LLM으로 정의한 다음, 이를 대화 체인에 전달합니다.

In [ ]:
import wget

if os.path.isfile(cwd + "/models/ggml-model-q4_0.gguf"):
    pass
else:
    wget.download("https://huggingface.co/davidkim205/komt-mistral-7b-v1-gguf/resolve/main/ggml-model-q4_0.gguf", out=cwd + "/models/")

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

n_gpu_layers = None  # Metal set to 1 is enough.
n_batch = 1  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    # https://huggingface.co/TheBloke/openbuddy-mistral-7B-v13-GGUF
    model_path = cwd + "/models/ggml-model-q4_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,

    # https://www.reddit.com/r/LocalLLaMA/comments/1343bgz/what_model_parameters_is_everyone_using/
    temperature=0.8,
    top_k=1,
    top_p=0.2,

    max_tokens=1024,
    verbose=True,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    repeat_penalty=1.1
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from lib.elasticsearch_chat_message_history import ElasticsearchChatMessageHistory
from uuid import uuid4

retriever = vector_store.as_retriever()

chat = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

session_id = str(uuid4())
chat_history = ElasticsearchChatMessageHistory(
    client=vector_store.client,
    session_id=session_id,
    index="workplace-docs-chat-history"
)

이제 챗봇에 질문을 할 수 있습니다!

각 질문에 대한 컨텍스트로 채팅 기록이 어떻게 전달되는지 확인하세요.

In [ ]:
# Define a convenience function for Q&A
def ask(question, chat_history):
    result = chat({"question": question, "chat_history": chat_history.messages})
    print(f"""[QUESTION] {question}
[ANSWER]  {result["answer"]}
          [SUPPORTING DOCUMENTS] {list(map(lambda d: d.metadata["name"], list(result["source_documents"])))}""")
    chat_history.add_user_message(result["question"])
    chat_history.add_ai_message(result["answer"])

# Chat away!
print(f"[CHAT SESSION ID] {session_id}")

💡 _Try experimenting with other questions or after clearing the workplace data, and observe how the responses change._


In [ ]:
ask("What does NASA stand for?", chat_history)


In [ ]:
ask("Which countries are part of it? And who are the team's leads?", chat_history)


한국어 질의는 답변을 적절히 처리하지 못합니다

In [ ]:
ask("NASA의 정의를 알려줘. 그리고 리더는 누구야?", chat_history)

# (Optional) Clean up 🧹

완료되면 이 세션의 채팅 기록을 정리할 수 있습니다

In [ ]:
chat_history.clear()

... or delete the indices.


In [ ]:
vector_store.client.indices.delete(index='workplace-docs')
vector_store.client.indices.delete(index='workplace-docs-chat-history')